In [10]:
import torch, gc
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
from torch.utils.data import Dataset

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision

from timeit import default_timer as timer


import os
from PIL import Image
import pickle

import numpy as np
import pandas as pd

class SingleFolderDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.image_list = os.listdir(folder_path)
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder_path, self.image_list[idx])
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)

        return image

In [11]:
# GPU
device = torch.device("cuda")
cpu_device = torch.device("cpu")

# Data Augmentation function

- Resize
- Center Crop
- HorizontalRandomFlip
- Normalization

In [12]:
# data augmentation for efficientnet
torch.manual_seed(1)
# the training transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225]),
])
train_transform_H = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225]),
])
# the validation transforms
valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225]),
])

# Read Data

In [13]:
BATCH_SIZE = 32
# training dataset
train_dataset = datasets.ImageFolder(
    root='train',
    transform=train_transform
)
train_dataset_H = datasets.ImageFolder(
    root='train',
    transform=train_transform_H
)
concatenated_dataset = ConcatDataset([train_dataset, train_dataset_H])

# validation dataset
valid_dataset = datasets.ImageFolder(
    root='valid',
    transform=valid_transform
)

test_dataset = SingleFolderDataset('test', transform=valid_transform)

# training data loaders
train_loader = DataLoader(
    concatenated_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=0, pin_memory=False
)
# validation data loaders
valid_loader = DataLoader(
    valid_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=0, pin_memory=False
)
# test
test_num = list()
for file_name in test_dataset.image_list:
    test_num.append(int(file_name.split('.')[0]))
test_indices = sorted(range(len(test_num)), key=lambda k: test_num[k])
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Try Pretrained EFFICIENTNET_Bx

In [14]:
import efficientnet_pytorch
# efficient net need to use this pre-trained model, or it performs very bad :(
class EfficientNetEncoderHead(nn.Module):
    def __init__(self, depth, num_classes):
        super(EfficientNetEncoderHead, self).__init__()
        self.depth = depth
        self.base = efficientnet_pytorch.EfficientNet.from_pretrained(f'efficientnet-b{self.depth}')
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.output_filter = self.base._fc.in_features
        self.classifier = nn.Linear(self.output_filter, num_classes)
        self.act_out = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.base.extract_features(x)
        x = self.avg_pool(x).squeeze(-1).squeeze(-1)
        x = self.classifier(x)
        x = self.act_out(x)
        return x
    
def train(model,
          criterion,
          optimizer,
          train_loader,
          valid_loader,
          save_file_name,
          max_epochs_stop=3,
          n_epochs=5,
          print_every=1,
          train_on_gpu = True,
          Device = None):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): cnn to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        train_loader (PyTorch dataloader): training dataloader to iterate through
        valid_loader (PyTorch dataloader): validation dataloader used for early stopping
        save_file_name (str ending in '.pt'): file path to save the model state dict
        max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
        n_epochs (int): maximum number of training epochs
        print_every (int): frequency of epochs to print training stats

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
        history (DataFrame): history of train and validation loss and accuracy
    """

    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        # Set to training
        model.train()
        start = timer()
        
        sample_of_train = 0

        # Training loop
        for ii, (data, target) in enumerate(train_loader):
            # Tensors to gpu
            if train_on_gpu:
                data, target = data.to(Device), target.to(Device)

            # Clear gradients
            optimizer.zero_grad()
            # Predicted outputs are log probabilities
            output = model(data)

            # Loss and backpropagation of gradients
            #loss = criterion(output, target)
            loss = criterion(output, target)
            loss.backward()
            #loss.requres_grad = True

            # Update the parameters
            optimizer.step()

            # Track train loss by multiplying average loss by number of examples in batch
            sample_of_train += data.size(0)
            train_loss += loss.item() * data.size(0)

            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)

            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete| Time:{timer() - start:.2f}| Training loss:{train_loss/sample_of_train:.4f}|acc:{train_acc/sample_of_train:.4f}.',
                end='\r')

        # After training loops ends, start validation
        else:
            model.epochs += 1

            # Don't need to keep track of gradients
            with torch.no_grad():
                # Set to evaluation mode
                model.eval()

                # Validation loop
                for data, target in valid_loader:
                    # Tensors to gpu
                    if train_on_gpu:
                        data, target = data.to(Device), target.to(Device)

                    # Forward pass
                    output = model(data)

                    # Validation loss
                    loss = criterion(output, target)
                    # Multiply average loss times the number of examples in batch
                    valid_loss += loss.item() * data.size(0)

                    # Calculate validation accuracy
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    # Multiply average accuracy times the number of examples
                    valid_acc += accuracy.item() * data.size(0)

                # Calculate average losses
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)

                # Calculate average accuracy
                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)

                history.append([train_loss, valid_loss, train_acc, valid_acc])

                # Print training and validation results
                if (epoch + 1) % print_every == 0:
                    print(
                        f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                    )
                    print(
                        f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
                    )

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch

                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epochs_no_improve >= max_epochs_stop:
                        print(
                            f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
                        )
                        total_time = timer() - overall_start
                        print(
                            f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                        )

                        # Load the best state dict
                        model.load_state_dict(torch.load(save_file_name))
                        # Attach the optimizer
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'train_acc',
                                'valid_acc'
                            ])
                        return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
    return model, history

# Training Model Process(Skip this block and use training weight which we had trained.)

In [15]:
num_classes = 100
learning_rate = 0.0005
num_epochs = 200

for i in range(5):
    model = EfficientNetEncoderHead(depth=i, num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    model, history = train(
        model,
        criterion,
        optimizer,
        train_loader,
        valid_loader,
        save_file_name=os.path.join('model_weight',f'efficientnet_b{i}'),
        max_epochs_stop=5,
        n_epochs=num_epochs,
        print_every=1,
        train_on_gpu = True,
        Device = device)

Loaded pretrained weights for efficientnet-b0
Starting Training from Scratch.

Epoch: 0	100.00% complete| Time:94.01| Training loss:3.9637|acc:0.7075.
Epoch: 0 	Training Loss: 3.9637 	Validation Loss: 3.7352
		Training Accuracy: 70.75%	 Validation Accuracy: 89.80%
Epoch: 1	100.00% complete| Time:63.56| Training loss:3.7137|acc:0.9241.
Epoch: 1 	Training Loss: 3.7137 	Validation Loss: 3.7234
		Training Accuracy: 92.41%	 Validation Accuracy: 91.20%
Epoch: 2	100.00% complete| Time:63.28| Training loss:3.6800|acc:0.9530.
Epoch: 2 	Training Loss: 3.6800 	Validation Loss: 3.7114
		Training Accuracy: 95.30%	 Validation Accuracy: 91.80%
Epoch: 3	100.00% complete| Time:63.04| Training loss:3.6721|acc:0.9581.
Epoch: 3 	Training Loss: 3.6721 	Validation Loss: 3.6922
		Training Accuracy: 95.81%	 Validation Accuracy: 93.80%
Epoch: 4	100.00% complete| Time:63.61| Training loss:3.6636|acc:0.9650.
Epoch: 4 	Training Loss: 3.6636 	Validation Loss: 3.6760
		Training Accuracy: 96.50%	 Validation Accuracy

# Prediction Test Data

In [16]:
num_classes = 100
prob = np.ones((5,500,100))

for i in range(5):
    model = EfficientNetEncoderHead(depth=i, num_classes=num_classes).to(device)
    model.load_state_dict(torch.load(os.path.join('model_weight',f'efficientnet_b{i}')))
    label_pred = np.ones((1,100))
    with torch.no_grad():
        model.eval()
        for images in test_loader:
            images = images.to(device)
            outputs = model(images).detach().cpu().numpy()
            label_pred = np.append(label_pred, outputs, axis=0)
    label_pred = label_pred[1:,:][test_indices]
    prob[i,:,:] = label_pred
    
with open(os.path.join('pred_prob','prob_efficient.pickle'), 'wb') as f:
    pickle.dump(prob, f)

Loaded pretrained weights for efficientnet-b0
Loaded pretrained weights for efficientnet-b1
Loaded pretrained weights for efficientnet-b2
Loaded pretrained weights for efficientnet-b3
Loaded pretrained weights for efficientnet-b4


# Calculate Accuracy

In [18]:
label_true = pd.read_csv('Test_label.csv')
with open(os.path.join('pred_prob','prob_efficient.pickle'), 'rb') as f:
    eff_prob = pickle.load(f)

name_eff = ['B0', 'B1', 'B2', 'B3', 'B4']


def acc_calculate(name_list,
                  prob_list,
                  label_true):
    model_n = len(name_list)
    acc = list()
    name = list()
    for i in range(model_n):
        prob = prob_list[i]
        prob = np.array(prob)
        prob_label = pd.DataFrame(prob.argmax(axis=1), columns=['Category'])
        acc.append(sum(prob_label['Category'] == label_true['Category'])/500)
        name.append("EfficientNet" + name_list[i])
    return_dict = {'model' : name, 'acc' : acc}
    return pd.DataFrame(return_dict)

accuracy = acc_calculate(name_eff,eff_prob,label_true)

print(accuracy)

            model    acc
0  EfficientNetB0  0.966
1  EfficientNetB1  0.964
2  EfficientNetB2  0.952
3  EfficientNetB3  0.952
4  EfficientNetB4  0.946
